In [86]:
import pandas as pd
import numpy as np
from datetime import datetime

## Pilots

In [102]:
n_pilots = 50
pilot_fields = ['sex', 'bd', 'p9', 'p10', 'p11', 'p12', 'p13', 'p14', 'trend']

In [103]:
pilots = pd.DataFrame(np.zeros((n_pilots, len(pilot_fields))), columns=pilot_fields)

In [104]:
pilots['sex'] = np.random.randint(0,2, n_pilots)

In [105]:
base_date = np.datetime64('1960-01-01')
pilots['bd'] = base_date + np.random.randint(0, 365*35, n_pilots)

In [106]:
pilots.iloc[:, 2:8] = np.random.randint(50, 71, (n_pilots, 6))

In [107]:
pilots['trend'] = np.random.randint(-3, 4, n_pilots)

In [108]:
#pilots

In [109]:
pilots.to_excel('Pilots.xlsx')

## Aircrafts

In [285]:
n_aircrafts = 2
aircrafts_fields = ['Name']

In [286]:
aircrafts = pd.DataFrame(np.zeros((n_aircrafts, len(aircrafts_fields))), columns=aircrafts_fields)

In [287]:
aircrafts_names = np.arange(100, 1000)
aircrafts['Name'] = np.random.choice(aircrafts_names, n_aircrafts, replace=False)

In [288]:
# aircrafts

In [289]:
aircrafts.to_excel('Aircrafts.xlsx')

## Flights

In [290]:
flight_fields = ['pilot_id', 'aircraft_id', 'date', 'weather', 'time', 'duration',
                 'sex', 'age', 'p9', 'p10', 'p11', 'p12', 'p13', 'p14']

In [295]:
f = pd.Series(index=flight_fields)

In [296]:
def rand(low, high):
    return np.random.random() * (high - low) + low

In [297]:
base_date = np.datetime64('2017-01-01')

flights = pd.DataFrame(columns=flight_fields)

for day_of_year in range(0, 366):
    date_repeat = np.random.randint(1, 2)
    curdate = base_date + day_of_year
    
    for i in range(date_repeat):
        f = pd.Series(index=flight_fields)
        
        f.pilot_id = np.random.choice(pilots.index.values)
        pilot = pilots.loc[f.pilot_id, :]
        
        f.aircraft_id = np.random.choice(aircrafts.index.values)
        f.date = curdate.__str__()
        f.weather = np.random.randint(0, 2)
        f.time = np.random.randint(0, 2)
        f.duration = np.random.randint(90, 601)
        
        f.sex = pilot['sex']
        birthdate = np.datetime64(pilot['bd'].__str__().split()[0])
        f.age = int((curdate - birthdate).astype('timedelta64[Y]').__str__().split()[0])
        
        trend = pilot['trend']
        
        for feature in ['p9', 'p10', 'p11']:
            f[feature] = (pilot[feature] 
                          + trend * day_of_year * rand(-0.005, 0.01)
                         )
            if f[feature] < 20:
                f[feature] = np.random.randint(10, 21)
            elif f[feature] > 95:
                f[feature] = np.random.randint(95, 101)
            else:
                f[feature] = f[feature].astype(int)
        
        for feature in ['p12', 'p13', 'p14']:
            f[feature] = (pilot[feature] 
                          + trend * day_of_year * rand(-0.005, 0.01)
                          + f['weather'] * rand(-5, 1)
                          + f['time'] * rand(-5, 1)
                          + f['duration'] * rand(-0.001, 0)
                          + f['sex'] * rand(-2, 5)
                          + f['age'] * rand(-0.05, 0.1)
                          + f['p9'] * rand(0, 0.03)
                          + f['p10'] * rand(0, 0.03)
                          + f['p11'] * rand(0, 0.03)
                         )
            if f[feature] < 20:
                f[feature] = np.random.randint(10, 21)
            elif f[feature] > 95:
                f[feature] = np.random.randint(95, 101)
            else:
                f[feature] = f[feature].astype(int)
                
        flights = flights.append(f, ignore_index=True)         
        

In [298]:
flights.head(5)

,pilot_id,aircraft_id,date,weather,time,duration,sex,age,p9,p10,p11,p12,p13,p14
0,46.0,1.0,2017-01-01,1,0,276,0,26,68,68,54,62,62,65
1,46.0,1.0,2017-01-02,0,0,268,0,26,67,67,53,66,61,72
2,5.0,0.0,2017-01-03,1,1,141,0,29,51,61,63,61,53,47
3,6.0,0.0,2017-01-04,0,0,390,1,32,66,68,51,57,66,66
4,46.0,0.0,2017-01-05,0,0,486,0,26,67,68,53,64,64,70


In [299]:
flights.to_excel('Flights.xlsx')